In [13]:
%pip install Tweepy

     |████████████████████████████████| 153kB 2.9MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [44]:
import tweepy
import json
import time


In [45]:
#Variable de autentificación en Twitter
OAUTH_TOKEN='1204477616398503936-7Z5WSIa0490S7HbIoPrDzosdA8aJKb'
OAUTH_SECRET='OCHWxZGQlrnaoNx4corTWHNZPrtGtItteYnow7nMDhtON'
CONSUMER_KEY='IRuah0diiQDx7ayuHaFmCOvTq'
CONSUMER_SECRET='FkYZE9vZXWMgWvovRjcqQaixPleh6ADXdD3XU05ZsIkscSFzXd'

In [46]:
auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN,OAUTH_SECRET)
api = tweepy.API(auth)

In [16]:
#Búsqueda de Tweets por palabra
def busqueda_tweets_palabra(palabra="Ryanair", lenguaje="es", numero_de_Tweets=200):
    return tweepy.Cursor(api.search, palabra, lang=lenguaje).items(numero_de_Tweets)

tweets=busqueda_tweets_palabra()
outtweets = [(tweet.created_at, 
              tweet.text, 
              tweet.retweet_count, 
              str(tweet.favorite_count)+'') for tweet in tweets]
outtweets

[(datetime.datetime(2019, 12, 18, 19, 3, 53),
  'Ryanair atribuye los recortes en Girona a los problemas del Boeing 737 MAX https://t.co/asx2LZmyV3 vía @elpais_economia',
  0,
  '0'),
 (datetime.datetime(2019, 12, 18, 19, 0, 8),
  'Los asientos separados, el nuevo mercadillo de las low cost (Ryanair, Vueling, etc...) https://t.co/Zg75Rhm4re',
  0,
  '0'),
 (datetime.datetime(2019, 12, 18, 18, 49, 18),
  'RT @marcosleyley: He visto sardinas en lata que tienen más espacio que volar en Vueling o Ryanair. Es curioso que mientras la población mun…',
  3,
  '0'),
 (datetime.datetime(2019, 12, 18, 18, 48, 17),
  'si tienes cualquier problema para que puedas resolverlo "perfectamente" te ponen un número de atención al cliente d… https://t.co/e94lP2nyZ5',
  0,
  '1'),
 (datetime.datetime(2019, 12, 18, 18, 40),
  'viajar con @Ryanair_ES y otras maneras de solo tener problemas.',
  0,
  '2'),
 (datetime.datetime(2019, 12, 18, 18, 36, 2),
  'Inspección de Trabajo abrirá expediente ✈️ https://t.co/

In [56]:
#El getter de Twitter contiene limitación al obtener los datos ya que devuelve 5000 cuando en realidad tiene 285055
def obtener_follower_ids():
    return api.followers_ids('@elcorteingles')

print(len(obtener_follower_ids()))
      

5000


In [65]:
def obtener_friends_ids():
    return api.friends_ids('@elcorteingles')

print(len(obtener_friends_ids()))

1631


In [62]:
#De aquí podemos obtener el número de followers que contiene la cuenta
def obtener_friends_ids():
    return api.get_user('@elcorteingles')
print(obtener_friends_ids())

User(_api=<tweepy.api.API object at 0x1073ee150>, _json={'id': 262541531, 'id_str': '262541531', 'name': 'El Corte Inglés', 'screen_name': 'elcorteingles', 'location': '', 'profile_location': None, 'description': 'Algunos viven momentos, nosotros los creamos: comparte con nosotros tu día a día. Envíanos tus consultas por MD a @elcorteinglesAC o menciónanos con #MyECI', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 285054, 'friends_count': 1631, 'listed_count': 1209, 'created_at': 'Tue Mar 08 07:19:56 +0000 2011', 'favourites_count': 128933, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': True, 'statuses_count': 74983, 'lang': None, 'status': {'created_at': 'Thu Dec 26 18:00:00 +0000 2019', 'id': 1210258979798110210, 'id_str': '1210258979798110210', 'text': 'El 27 de junio de 2020, revive las canciones más míticas de la primera década de los 2000 con el festival… https://t.co/z01UamixDW', 'truncated': True, 'entiti

In [81]:
ids,cursor = api.friends_ids('@elcorteingles',cursor=-1, count=5000)
print(ids)

[182764833, 200081186, 1042007718553415682, 197399746, 5494572, 1728956976, 1095345891542736897, 403396902, 62591415, 14712261, 1042012837319770114, 981893453142548480, 288787209, 505763621, 1097413622517690370, 934833810, 167481898, 170819033, 16684359, 2827858225, 383294819, 847017228033806336, 4044880565, 1734700669, 334684429, 2251552290, 2978762543, 177559871, 2269563059, 4888068429, 2174698591, 97654669, 2174859445, 17209618, 92281182, 1060678345808068608, 955613952, 804617603012501504, 259824220, 1946527747, 827527807, 2172740569, 283699138, 2989465757, 1047037402215268352, 306112241, 42050597, 198787442, 1379710231, 3392188757, 978000108183834624, 1951504608, 117748602, 574649962, 889307215, 1132246848, 202092114, 1501434991, 364968471, 2480087774, 921689998507827202, 986550461850443778, 856451172416311296, 2183187386, 1024860452, 18736723, 1018495825, 1325475392, 710144100679421953, 3378657623, 304538396, 296374641, 979291508, 140788235, 243127136, 2873314776, 224645268, 57430

In [82]:
type(ids)

list

In [ ]:
#Obtención de las cuentas que sigue El Corte Inglés
def get_info_friends(limit=5000):
    cursor = -1
    friends_list=list()
    while cursor != 0:
        ids_friends,cursor_friends = api.friends_ids('@elcorteingles', cursor=cursor, count=limit)
        friends_list = ids_friends
        cursor = cursor_friends
        time.sleep(60)
    return friends_list

lista_friends=get_info_friends()
print(lista_friends)

In [78]:
#Obtención de las cuentas que siguen El Corte Inglés
def get_info_followers(limit=5000):
    cursor = -1
    followers_list=list()
    while cursor != 0:
        ids_follow,cursor_followers = api.followers_ids('@elcorteingles', cursor=cursor, count=limit)
        followers_list = followers_list.append(ids_follow)
        cursor = cursor_followers
        time.sleep(60)
    return followers_list

lista_followers= get_info_followers()

AttributeError: 'NoneType' object has no attribute 'append'